# 컴퓨터 비전 기초: 합성곱 신경망

## 주요 내용

- 합성곱 신경망

- 데이터 증식

- 모델 재활용: 전이학습

## 합성곱 신경망

- 2011년부터 2015년: 컴퓨터 비전 분야에서 딥러닝 기법이 획기적으로 발전

- 사진 검색, 자율주행, 로봇공학, 의학 진단 프로그램,얼굴 인식 등 일상의 많은 영역에서 활용됨

- 컴퓨터 비전 분야 딥러닝 모델: **CNN** 또는 **convnet**으로 불리는 **합성곱 신경망**이 대세

### MNIST 데이터셋 분류 CNN 모델

```python
# 입력층
inputs = keras.Input(shape=(28, 28, 1))

# 은닉층
x = layers.Conv2D(filters=32, kernel_size=3, activation="relu")(inputs)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=64, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=128, kernel_size=3, activation="relu")(x)

# 출력층으로 넘기기 전에 1차원 텐서로 변환
x = layers.Flatten()(x)

# 출력층
outputs = layers.Dense(10, activation="softmax")(x)

# 모델
model = keras.Model(inputs=inputs, outputs=outputs)
```

### MNIST 이미지 분류 훈련

- 훈련셋 준비

```python
from tensorflow.keras.datasets import mnist

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
train_images = train_images.reshape((60000, 28, 28, 1))
train_images = train_images.astype("float32") / 255
test_images = test_images.reshape((10000, 28, 28, 1))
test_images = test_images.astype("float32") / 255
```

- 모델 컴파일과 훈련

```python
model.compile(optimizer="rmsprop",
    loss="sparse_categorical_crossentropy",  # 레이블이 정수인 경우
    metrics=["accuracy"])

model.fit(train_images, train_labels, epochs=5, batch_size=64)
```

### 합성곱 연산

- `Dense` 층: 입력값의 전체 특성을 대상으로 한 번의 아핀 변환 적용

- `Conv2D` 층: `kernel_size`로 지정된 크기의 공간에 대해 여러 개의 아핀 변환 적용.
    예를 들어, `kernel_size=3`인 경우 `3x3` 크기의 영역에 대해 아핀 변환 적용.

- `Dense` 층: 입력값 전체를 대상으로 지정된 개수의 좋은 특성으로 구성된 텐서로 데이터 변환을 진행한다.
    예를 들어, `Dense(64, activation="relu")`에 MNIST 데이터셋을 입력하면
    흑백 손글씨 이미지 각각에 대해 784개 특성을 아핀 변환하여 64개의 좋은 특성을 생성한다.

- `Conv2D` 층: 예를 들어 `kernel_size=3`으로 설정된 경우 `3x3` 크기의 영역에 대해 국소적 패턴을 찾아낸다. 
    그리고 그런 국소적 패턴들을 종합하여 모델 예측에 도움되는 데이터로 변환한다.

### 합성곱 층의 특징

첫째, 위치와 무관하게 패턴을 찾아낸다.
즉, 서로 다른 위치에 있는 동일한 패턴은 동일한 방식으로 인식된다.

<div align="center"><img src="https://drek4537l1klr.cloudfront.net/chollet2/v-7/Figures/local_patterns.jpg" style="width:300px;"></div>

둘째, **패턴 공간의 계층**<font size='2'>spatial hierarchy of patterns</font> 파악

<div align="center"><img src="https://drek4537l1klr.cloudfront.net/chollet2/v-7/Figures/visual_hierarchy_hires.png" style="width:400px;"></div>

### 컬러 이미지와 채널

<div align="center"><img src="https://raw.githubusercontent.com/codingalzi/dlp2/master/jupyter-book/imgs/ch08-reign_pic_breakdown.png" style="width:300px;"></div>

<div align="center"><img src="https://raw.githubusercontent.com/codingalzi/dlp2/master/jupyter-book/imgs/ch08-three_d_array-a.jpg" style="width:400px;"></div>

### 특성맵(채널), 필터, 출력맵

- **특성맵**<font size='2'>feature map</font> 또는 **채널**<font size='2'>channel</font>:
    - `(높이, 너비)` 모양의 2D 텐서.
    - 예제: MNIST 데이터셋에 포함된 흑백 이미지 샘플의 경우 `(28, 28)` 모양의 채널(특성맵) 한 개로 구성됨.
    - 예제: 컬러사진의 경우 세 개의 채널(특성맵)로 구성됨.
    - 이미지에 포함된 채널(특성맵)의 수를 **깊이**라 부름.
- **필터**<font size='2'>filter</font>: `kernel_size`를 이용한 3D 텐서. 
    - 예제: `kernel_size=3`인 경우 필터는 `(3, 3, 입력샘플의깊이)` 모양의 3D 텐서.
    - 필터 수: `filters` 인자에 의해 결정됨.
- **출력맵**<font size='2'>response map</font>: 
    입력 샘플을 대상으로 하나의 필터를 적용해서 생성된 하나의 특성맵(채널).
    필터 수만큼의 출력맵 생성.

<div align="center"><img src="https://raw.githubusercontent.com/codingalzi/dlp2/master/jupyter-book/imgs/ch08-three_d_array-a.jpg" style="width:400px;"></div>

### 필터 적용

<div align="center"><img src="https://raw.githubusercontent.com/codingalzi/dlp2/master/jupyter-book/imgs/ch08-filter-product-1c.jpg" style="width:700px;"></div>

<p><div style="text-align: center">&lt;그림 출처: <a href="https://www.oreilly.com/library/view/fundamentals-of-deep/9781492082170/">Fundamentals of Deep Learning(2판)</a>&gt;</div></p>

### 출력맵

<div align="center"><img src="https://raw.githubusercontent.com/codingalzi/dlp2/master/jupyter-book/imgs/ch08-convSobel-2a.gif" style="width:300px;"></div>

```
0 * -1  + 0  * -2 + 75 * -1 +
0 *  0  + 75 *  0 + 80 *  0 +
0 *  1  + 75 *  2 + 80 *  1 +
0
= 155
```

### 필터와 출력맵

- 입력 텐서: (10, 8, 3) 모양의 텐서
- 필터: (3, 3, 3) 모양의 텐서
    - 커널 크기(`kernel_size`): 3
    - 입력 텐서의 깊이: 3
- 출력 텐서: (8, 6, 6) 모양의 텐서
    - 필터 수가 6이기에 출력 텐서의 깊이가 6이 됨.

<div align="center"><img src="https://raw.githubusercontent.com/codingalzi/dlp2/master/jupyter-book/imgs/ch08-filter-product-2a.jpg" style="width:400px;"></div>

### 합성곱 층 연속 적용

<div align="center"><img src="https://raw.githubusercontent.com/codingalzi/dlp2/master/jupyter-book/imgs/ch08-cnn-layers-a.jpg" style="width:500px;"></div>

### 패딩과 보폭

- 경우 1: 패딩 없음, 보폭은 1.
    - 필터라 1칸씩 슬라이딩 함.
    - 출력 특성맵의 깊이와 너비: `3x3`
    - 출력 특성맥의 깊이와 너비가 줄어듦.

<div align="center"><img src="https://raw.githubusercontent.com/codingalzi/dlp2/master/jupyter-book/imgs/ch08-padding-stride-01.png" style="width:400px;"></div>

- 경우 2: 패딩 없음, 보폭은 2.
    - 필터라 2칸씩 건너 뛰며 슬라이딩 함.
    - 출력 특성맵의 깊이와 너비: `2x2`
    - 출력 특성맵의 깊이와 너비가 보폭의 반비례해서 줄어듦.

<div align="center"><img src="https://raw.githubusercontent.com/codingalzi/dlp2/master/jupyter-book/imgs/ch08-padding-stride-02.png" style="width:350px;"></div>

- 경우 3: 패딩 있음, 보폭은 1.
    - 입력 텐서의 테두리에 0으로 채워진 패딩 추가.
    - 출력 특성맵의 깊이와 너비가 동일하게 유지됨.

<div align="center"><img src="https://raw.githubusercontent.com/codingalzi/dlp2/master/jupyter-book/imgs/ch08-cnn-padding.png" style="width:700px;"></div>

### 맥스 풀링

<div align="center"><img src="http://formal.hknu.ac.kr/handson-ml2/slides/images/ch14/homl14-03.png" style="width:700px;"></div>

<p><div style="text-align: center">&lt;그림 출처: <a href="https://www.hanbit.co.kr/store/books/look.php?p_code=B7033438574">핸즈온 머신러닝(2판)</a>&gt;</div></p>

### 맥스 풀링 기능

```python
layers.MaxPooling2D(pool_size=2)(x)
```

<div align="center"><img src="http://formal.hknu.ac.kr/handson-ml2/slides/images/ch14/homl14-10.png" style="width:600px;"></div>

### 맥스 풀링 사용 이유

- 모델이 훈련 중에 학습해야할 파라미터(가중치와 편향)의 수를 줄인다.
    - 합성곱 층 자체에서 사용되는 파라미터의 수는 맥스 풀링 층에 줄어들지 않는다.
    - 반면에 합성곱 층에 이어서 사용되는 `Dense` 층의 입력값에 갖는 특성 수가 획기적으로 줄어든다.

- 상위 합성곱 층으로 이동 수록 입력 데이터의 보다 넓은 영역에 대한 정보를 얻을 수 있다.

## 합성곱 신경망 실전 활용 예제

### 작은 데이터셋과 딥러닝 모델

- 이미지 분류 모델을 훈련시킬 때 데이터셋의 크기가 그다지 크지 않은 경우가 일반적이다.

- 여기서 훈련시켜야 하는 모델은 개와 고양이 사진을 대상으로 하는 이진 분류 합성곱 신경망 모델이다.

- 실전 상황을 재현하기 위해 5천 개의 이미지로 이루어진 작은 데이터셋을 사용한다.

### 데이터 다운로드

- 캐글 계정을 갖고 있어야 하며, 로그인된 상태에서 아래 두 과정을 먼저 해결해야 한다.

- 캐글에 로그인한 후 "Account" 페이지의 계정 설정 창에 있는 "API" 항목에서
    "Create New API Token"을 생성하여 다운로드한다.

- [캐글: Dogs vs. Cats](https://www.kaggle.com/c/dogs-vs-cats/rules)를
    방문해서 "I Understand and Accept" 버튼을 클릭해야 한다.

- 총 25,000장의 강아지와 고양이 사진으로 구성되었으며 570MB 정도로 꽤 크다.
    - 강아지 사진 고양이 사진이 각각 12,500 장씩 포함되어 있으며, 사진들의 크기가 다음과 같이 일정하지 않다.

<div align="center"><img src="https://drek4537l1klr.cloudfront.net/chollet2/v-7/Figures/dog_and_cat_samples.png" style="width:500px;"></div>

### 훈련셋, 검증셋, 테스트셋 준비

25,000 장의 사진 중에서 총 5,000 장의 사진만 사용해서 합성곱 신경을 훈련시키려 한다.

- 훈련셋: 강아지와 고양이 각각 1,000 장
- 검증셋: 강아지와 고양이 각각 500 장
- 테스트셋: 강아지와 고양이 각각 1,000 장

```
cats_vs_dogs_small/
...train/
......cat/
......dog/
...validation/
......cat/
......dog/
...test/
......cat/
......dog/
```

### 모델 지정

```python
# 입력층
inputs = keras.Input(shape=(180, 180, 3))

# 은닉층
x = layers.Rescaling(1./255)(inputs)
x = layers.Conv2D(filters=32, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=64, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=128, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=256, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=256, kernel_size=3, activation="relu")(x)
x = layers.Flatten()(x)

# 출력층
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs=inputs, outputs=outputs)
```

### 모델 컴파일

```python
model.compile(loss="binary_crossentropy",
              optimizer="rmsprop",
              metrics=["accuracy"])
```

### 데이터 전처리

`cats_vs_dogs_small`의 하위 디렉토리인 `train`, `validation`, `test` 디렉토리에 포함된
사진들을 무작위로 섞어 크기가 32인 배치들로 구성된 훈련셋, 검증셋, 테스트셋을 지정한다.

```python
from tensorflow.keras.utils import image_dataset_from_directory

new_base_dir = pathlib.Path("cats_vs_dogs_small")

train_dataset = image_dataset_from_directory(
    new_base_dir / "train",
    image_size=(180, 180),
    batch_size=32)

validation_dataset = image_dataset_from_directory(
    new_base_dir / "validation",
    image_size=(180, 180),
    batch_size=32)

test_dataset = image_dataset_from_directory(
    new_base_dir / "test",
    image_size=(180, 180),
    batch_size=32)
```

### 모델 훈련

크기가 32인 배치 단위로 이미 묶여 있기에 `fit()` 메서드를 호출할 때 배치 크기(`batch_size`)는 지정할 필요가 없다.

```python
history = model.fit(
    train_dataset,
    epochs=30,
    validation_data=validation_dataset,
    callbacks=callbacks)
```

과대 적합이 10번 정도의 에포크 이후에 빠르게 발생한다.

<div align="center"><img src="https://drek4537l1klr.cloudfront.net/chollet2/Figures/08-09.png" style="width:800px;"></div>

<p><div style="text-align: center">&lt;그림 출처: <a href="https://www.manning.com/books/deep-learning-with-python-second-edition">Deep Learning with Python(2판)</a>&gt;</div></p>

### 데이터 증식

- `RandomFlip()`: 사진을 50%의 확률로 지정된 방향으로 반전. 
- `RandomRotation()`: 사진을 지정된 범위 안에서 임의로 좌우로 회전
- `RandomZoom()`: 사진을 지정된 범위 안에서 임의로 확대 및 축소

```python
data_augmentation = keras.Sequential(
    [layers.RandomFlip("horizontal"),
     layers.RandomRotation(0.1),
     layers.RandomZoom(0.2)]
)
```

훈련셋의 이미지 샘플 하나를 대상으로 데이터 증식 층을 아홉 번 적용한 결과를 
다음고 같다.

<div align="center"><img src="https://drek4537l1klr.cloudfront.net/chollet2/Figures/08-10.png" style="width:400px;"></div>

```python
inputs = keras.Input(shape=(180, 180, 3))
x = data_augmentation(inputs)
x = layers.Rescaling(1./255)(x)
x = layers.Conv2D(filters=32, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=64, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=128, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=256, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=256, kernel_size=3, activation="relu")(x)
x = layers.Flatten()(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs=inputs, outputs=outputs)
```

과대 적합이 보다 늦게 발생한다.

<div align="center"><img src="https://drek4537l1klr.cloudfront.net/chollet2/Figures/08-11.png" style="width:800px;"></div>

<p><div style="text-align: center">&lt;그림 출처: <a href="https://www.manning.com/books/deep-learning-with-python-second-edition">Deep Learning with Python(2판)</a>&gt;</div></p>

## 모델 재활용

적은 양의 데이터셋을 대상으로 훈련하는 것보다 대용량의 데이터셋을 이용하여 훈련하면
보다 좋은 성능의 모델을 구현할 수 있다.
하지만 대용량의 데이터를 구하기는 매우 어렵거나 아예 불가능할 수 있다.
하지만 유사한 목적으로 대용량의 훈련 데이터셋을 이용하여 사전에 훈련된 모델을 재활용하면
높은 성능의 모델을 얻을 수 있다.

여기서는 좋은 성능으로 잘 알려진 모델인 VGG16을 재활용하여 높은 성능의 
강아지와 고양이 분류 모델을 구현하는 두 가지 방식을 소개한다.

- 전이 학습<font size='2'>transfer learning</font>
- 모델 미세조정<font size='2'>model fine tuning</font>

**VGG16 모델**

VGG16 모델은 [ILSVRC 2014](https://www.image-net.org/challenges/LSVRC/2014/) 
경진대회에 참여해서 2등을 차지한 모델이다.
당시 훈련에 사용된 데이터셋은 120만 장의 이미지와 1,000개의 클래스로 구성되었으며
훈련은 여러 주(weeks)에 걸쳐서 진행되었다. 

<div align="center"><img src="https://www.image-net.org/static_files/figures/ILSVRC2012_val_00042692.png" style="width:500px;"></div>

<div align="center"><img src="https://neurohive.io/wp-content/uploads/2018/11/vgg16-1-e1542731207177.png" style="width:500px;"></div>

**유명 합성곱 신경망 모델**

`ketas.applications` 에 포함된 유명 합성곱 신경모델은 다음과 같다.

- VGG16
- Xception
- ResNet
- MobileNet
- EfficientNet
- DenseNet
- 등등

### 이미지넷(ImagNet) 소개

- 이미지넷(Imagenet)](https://www.image-net.org/index.php): 대용량의 이미지 데이터셋
- [ILSVRC](https://www.image-net.org/challenges/LSVRC/index.php) 이미지 분류 경진대회에 사용되었음.
- 총 2만2천 개 정도의 클래스로 구분되는 동물, 사물 등의 객체를 담은 고화질 사진 1500만장 정도

<div align="center"><img src="https://cs.stanford.edu/people/karpathy/cnnembed/cnn_embed_full_1k.jpg" style="width:50%;"></div>

### 전이 학습

<div align="center"><img src="https://drek4537l1klr.cloudfront.net/chollet2/Figures/08-12.png" style="width:500px;"></div>

### VGG16 모델을 이용한 전이 학습

```python
conv_base = keras.applications.vgg16.VGG16(
    weights="imagenet",
    include_top=False,
    input_shape=(180, 180, 3))
```

모델을 가져올 때 사용된 옵션의 의미는 다음과 같다.

- `weights="imagenet"`: Imagenet 데이터셋으로 훈련된 모델의 가중치 가져옴.
- `include_top=False`: 출력값을 결정하는 밀집층은 제외함.
- `input_shape=(180, 180, 3)`: 앞서 준비해 놓은 데이터셋을 활용할 수 있도록 지정함. 사용자가 직접 지정해야 함.
    지정하지 않으면 임의의 크기의 이미지를 처리할 수 있음. 
    층 별 출력 텐서의 모양의 변화 과정을 확인하기 위해 특정 모양으로 지정함.

```python
>>> conv_base.summary()
Model: "vgg16" 
_________________________________________________________________
Layer (type)                 Output Shape              Param # 
=================================================================
input_19 (InputLayer)        [(None, 180, 180, 3)]     0 
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 180, 180, 64)      1792 
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 180, 180, 64)      36928 
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 90, 90, 64)        0 
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 90, 90, 128)       73856 
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 90, 90, 128)       147584 
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 45, 45, 128)       0 
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 45, 45, 256)       295168 
_________________________________________________________________
block3_conv2 (Conv2D)        (None, 45, 45, 256)       590080 
_________________________________________________________________
block3_conv3 (Conv2D)        (None, 45, 45, 256)       590080 
_________________________________________________________________
block3_pool (MaxPooling2D)   (None, 22, 22, 256)       0 
_________________________________________________________________
block4_conv1 (Conv2D)        (None, 22, 22, 512)       1180160 
_________________________________________________________________
block4_conv2 (Conv2D)        (None, 22, 22, 512)       2359808 
_________________________________________________________________
block4_conv3 (Conv2D)        (None, 22, 22, 512)       2359808 
_________________________________________________________________
block4_pool (MaxPooling2D)   (None, 11, 11, 512)       0 
_________________________________________________________________
block5_conv1 (Conv2D)        (None, 11, 11, 512)       2359808 
_________________________________________________________________
block5_conv2 (Conv2D)        (None, 11, 11, 512)       2359808 
_________________________________________________________________
block5_conv3 (Conv2D)        (None, 11, 11, 512)       2359808 
_________________________________________________________________
block5_pool (MaxPooling2D)   (None, 5, 5, 512)         0 
=================================================================
Total params: 14,714,688 
Trainable params: 14,714,688 
Non-trainable params: 0 
```

### 특성 추출

**특성 추출**<font size='2'>feature extraction</font>은 전이 학습에 사용되는 모델을 이용하여
데이터를 변환하는 과정을 의미한다.
여기서는 `conv_base` 기저를 특성 추출에 활용하는 두 가지 방식을 소개한다.

### 1) 단순 특성 추출

아래 `get_features_and_labels()` 함수는 
`conv_base` 모델의 `predict()` 메서드를 이용하여 
준비된 훈련 데이터셋을 변환, 
즉 특성 추출을 실행한다.
단, 레이블은 그대로 둔다.

- `keras.applications.vgg16.preprocess_input()` 함수는 텐서플로우와 호환이 되도록 데이터를 전처리한다.

```python
def get_features_and_labels(dataset):
    all_features = []
    all_labels = []
    
    # 배치 단위로 VGG16 모델 적용
    for images, labels in dataset:
        preprocessed_images = keras.applications.vgg16.preprocess_input(images)
        features = conv_base.predict(preprocessed_images)
        all_features.append(features)
        all_labels.append(labels)
        
    # 생성된 배치를 하나의 텐서로 묶어서 반환
    return np.concatenate(all_features), np.concatenate(all_labels)
```

훈련셋, 검증셋, 테스트셋을 변환하면 다음과 같다.

```python
train_features, train_labels =  get_features_and_labels(train_dataset)
val_features, val_labels =  get_features_and_labels(validation_dataset)
test_features, test_labels =  get_features_and_labels(test_dataset)
```

변환된 데이터셋을 훈련 데이터셋으로 사용하는 
간단한 분류 모델을 구성하여 훈련만 하면 된다.
`Dropout` 층은 과대적합을 예방하기 위해 사용한다.

```python
# 입력층
inputs = keras.Input(shape=(5, 5, 512))

# 은닉층
x = layers.Flatten()(inputs)
x = layers.Dense(256)(x)
x = layers.Dropout(0.5)(x)

# 출력층
outputs = layers.Dense(1, activation="sigmoid")(x)

# 모델
model = keras.Model(inputs, outputs)
```

검증셋에 대한 정확도가 97% 정도까지 향상되지만 과대적합이 매우 빠르게 발생한다.

<div align="center"><img src="https://drek4537l1klr.cloudfront.net/chollet2/Figures/08-13.png" style="width:700px;"></div>

<p><div style="text-align: center">&lt;그림 출처: <a href="https://www.manning.com/books/deep-learning-with-python-second-edition">Deep Learning with Python(2판)</a>&gt;</div></p>

### 2) 데이터 증식과 특성 추출

데이터 증식 기법을 활용하려면 
VGG16 합성곱 기저(베이스)를 구성요소로 사용하는 모델을 직접 정의해야 한다. 
다만 앞서 설명한 방식과는 달리 가져온 VGG16 기저에 포함된 파라미터가 새로운
모델의 훈련 과정동안 함께 훈련되지 않도록 **동결**(freezing)해야 한다.

- 기저 동결하기: `trainable=False`로 지정.
- 입력 데이터의 모양도 미리 지정하지 않음에 주의할 것.

```python
conv_base  = keras.applications.vgg16.VGG16(
    weights="imagenet",
    include_top=False)

# 새로운 학습 금지 설정
conv_base.trainable = False
```

```python
data_augmentation = keras.Sequential(
    [
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(0.1),
        layers.RandomZoom(0.2),
    ]
)

# 모델 구성
inputs = keras.Input(shape=(180, 180, 3))

x = data_augmentation(inputs)                     # 데이터 증식
x = keras.applications.vgg16.preprocess_input(x)  # VGG16용 전처리
x = conv_base(x)                                  # VGG16 베이스
x = layers.Flatten()(x)
x = layers.Dense(256)(x)
x = layers.Dropout(0.5)(x)

outputs = layers.Dense(1, activation="sigmoid")(x) # 출력층

model = keras.Model(inputs, outputs)
```

<div align="center"><img src="https://drek4537l1klr.cloudfront.net/chollet2/Figures/08-14.png" style="width:700px;"></div>

<p><div style="text-align: center">&lt;그림 출처: <a href="https://www.manning.com/books/deep-learning-with-python-second-edition">Deep Learning with Python(2판)</a>&gt;</div></p>

### 모델 미세 조정

<div align="center"><img src="https://drek4537l1klr.cloudfront.net/chollet2/Figures/08-15.png" style="width:100px;"></div>

아래 코드는 모든 층에 대해 동결해제를 진행한 후에
마지막 4개 층을 제외한 나머지 층에 대해 다시 동결을 설정한다.

```python
conv_base.trainable = True
for layer in conv_base.layers[:-4]:
    layer.trainable = False
```

```python
model.compile(loss="binary_crossentropy",
              optimizer=keras.optimizers.RMSprop(learning_rate=1e-5),
              metrics=["accuracy"])

callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath="fine_tuning",
        save_best_only=True,
        monitor="val_loss")
]
history = model.fit(
    train_dataset,
    epochs=30,
    validation_data=validation_dataset,
    callbacks=callbacks)
```

## 연습문제

1. [(실습) 컴퓨터 비전 기초: 합성곱 신경망](https://colab.research.google.com/github/codingalzi/dlp2/blob/master/excs/exc-computer_vision_intro.ipynb)